In [18]:
import sys
import os
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from bs4 import BeautifulSoup

In [10]:
def load_page_with_timeout(driver, url : str, timeout=5):
    driver.set_page_load_timeout(timeout)
    try:
        driver.get(url)
    except:
        driver.execute_script("window.stop();")

In [1]:
from selenium import webdriver

def get_chrome(headless :bool = False, adblock_extenstion_path : str = None ) -> webdriver.Chrome :
    """
    Initializes and returns a Chrome WebDriver instance.

    Args:
        headless (bool, optional): Whether to run in headless mode. Defaults to False.
        adblock_extenstion_path (str, optional): The path to the adblock extension. Defaults to None.

    Returns:
        webdriver.Chrome: A Chrome WebDriver instance.
    """
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    if adblock_extenstion_path != None :
        options.add_extension(adblock_extenstion_path)
    if headless :
        options.add_argument('--headless')
    options.add_argument('--blink-settings=imagesEnabled=false')
    options.add_argument("--log-level=3")

    return webdriver.Chrome( options=options )

In [11]:
driver = get_chrome(headless=False)
load_page_with_timeout(driver, "https://www.google.com/search?q=python+download", timeout=5)

In [20]:
response = requests.get('https://www.op.gg/champions?position=top&tier=challenger')

In [23]:
response

<Response [200]>

In [24]:
data = BeautifulSoup(response.text).find('table')

In [ ]:
all_tiers = ['challenger', 'master_plus', 'master_plus', 'master', 'diamond_plus', 'diamond', 'emerald_plus', 'master', 'platinum_plus', 'platinium', 'gold_plus', 'gold', 'silver_plus', 'silver', 'bronze_plus', 'bronze', 'iron_plus', 'iron']
all_roles = ['top', 'jungle', 'mid', 'adc', 'support']
all_patches = ['15.04']

In [49]:
def get_role_rank_patch_winrate(role, rank, patch):
    response = requests.get(f'https://www.op.gg/champions?position={role}&tier={rank}&patch={patch}')
    data = BeautifulSoup(response.text).find('table')

    rows = data.find('tbody').find_all('tr', class_ = "")
    rows = [row for row in rows]

    champions = []
    winrates = []
    pick_rates = []

    for row in rows:
        champions.append(row.find_all('td')[1].find('a').text.strip())
        winrates.append(float(row.find_all('td')[4].text.strip().replace('%', '')) / 100)
        pick_rates.append(float(row.find_all('td')[5].text.strip().replace('%', '')) / 100) 

    df = pd.DataFrame({'champion': champions, 'winrate': winrates, 'pick_rate': pick_rates})
    return df

In [58]:
role_rank_patch_winrate = get_role_rank_patch_winrate('top', 'gold', '15.04')

In [65]:
role_rank_patch_winrate.winrate

0     0.5212
1     0.5248
2     0.5095
3     0.5172
4     0.5258
5     0.5209
6     0.5293
7     0.5192
8     0.5256
9     0.5271
10    0.5096
11    0.5146
12    0.5004
13    0.5074
14    0.5128
15    0.5052
16    0.5023
17    0.5004
18    0.5042
19    0.5044
20    0.5204
21    0.4913
22    0.5045
23    0.4997
24    0.5060
25    0.5150
26    0.5182
27    0.4984
28    0.5028
29    0.5080
30    0.4838
31    0.4795
32    0.4958
33    0.4903
34    0.5151
35    0.4926
36    0.4936
37    0.5205
38    0.4927
39    0.5106
40    0.5186
41    0.5018
42    0.4909
43    0.4892
44    0.4771
45    0.4987
46    0.5125
47    0.4877
48    0.4886
49    0.5103
50    0.4903
51    0.4656
52    0.5038
53    0.4922
54    0.4927
Name: winrate, dtype: float64

In [62]:
role_rank_patch_winrate.pick_rate.sum()

1.8074000000000001